In [2]:
import pandas as pd
data = pd.read_csv('train.csv') # Arquivo do Titanic
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [3]:
y = data['Survived']
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [4]:
X = data.drop('Survived',axis=1) # Coloca todas as colunas no X menos o que já foi colocado em y (Survived)
X.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
for column in X.columns:
    print(f"{column:>12}: {len(set(X[column])):4} {X[column].dtype}") #Imprime o nome da coluna (column:>12). Quantos elementos tem repetitidos em cada coluna (len(set(X[column])) e o tipo de cada coluna (X[column].dtype)

 PassengerId:  891 int64
      Pclass:    3 int64
        Name:  891 object
         Sex:    2 object
         Age:  265 float64
       SibSp:    7 int64
       Parch:    7 int64
      Ticket:  681 object
        Fare:  248 float64
       Cabin:  148 object
    Embarked:    4 object


In [6]:
indesejadas = ['PassengerId', 'Name', 'Ticket', 'Cabin'] # Seleciona as características que não serão utilizadas
Xdrop = X.drop(indesejadas,axis=1) # Crua Xdrop como o X sem as colunas indesejadas
Xdrop.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [7]:
Xnum = Xdrop.select_dtypes('number') # Separando somente as características que possuem tipo numéricos
Xnum.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [8]:
for column in Xnum.columns:
    print(f"{column:>12}: {sum(Xnum[column].isnull())}") # Fazer o cálculo de quantos campos estão vazios (Xnum[column].isnull())

      Pclass: 0
         Age: 177
       SibSp: 0
       Parch: 0
        Fare: 0


In [9]:
from sklearn.impute import SimpleImputer # Itilizado para preencher os 177 valores nulos em idade
imputer = SimpleImputer(strategy='median') # Será feito uma mediana dos valores de idade e preenchido os valores nulos
XnumLimpo = imputer.fit_transform(Xnum)# Comando para executar o que foi programado em Xnum
XnumLimpo

array([[ 3.    , 22.    ,  1.    ,  0.    ,  7.25  ],
       [ 1.    , 38.    ,  1.    ,  0.    , 71.2833],
       [ 3.    , 26.    ,  0.    ,  0.    ,  7.925 ],
       ...,
       [ 3.    , 28.    ,  1.    ,  2.    , 23.45  ],
       [ 1.    , 26.    ,  0.    ,  0.    , 30.    ],
       [ 3.    , 32.    ,  0.    ,  0.    ,  7.75  ]])

In [10]:
Xcat = Xdrop.select_dtypes('object') # Fazendo a mesma coisa para os atributos categóricos
Xcat.columns

Index(['Sex', 'Embarked'], dtype='object')

In [11]:
for column in Xcat.columns:
    print(f"{column:>12}: {sum(Xcat[column].isnull())}") # Fazendo a soma de todos os valores que estiverem vazios

         Sex: 0
    Embarked: 2


In [12]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent') # Configurando como será o preenchimento dos valores vazios, no caso serão preenchidos com os valores mais frequentes
XcatLimpo = imputer.fit_transform(Xcat) # Comando para executar o que foi programado em Xcat
XcatLimpo

array([['male', 'S'],
       ['female', 'C'],
       ['female', 'S'],
       ...,
       ['female', 'S'],
       ['male', 'C'],
       ['male', 'Q']], dtype=object)

In [13]:
from sklearn.preprocessing import OneHotEncoder # Uma forma de pré-processar 
encoder = OneHotEncoder()
XcatHot = encoder.fit_transform(XcatLimpo) # Realizando o pré-processamento
XcatHot # Agora ele retorna uma matriz "sparsa", antes tinham 2 colunas e agora tem 5. 2 para o sexo (male e famale) e 3 para Embarked (S, C e Q)

<891x5 sparse matrix of type '<class 'numpy.float64'>'
	with 1782 stored elements in Compressed Sparse Row format>

In [14]:
import numpy as np 
Xtratado = np.c_[XnumLimpo,XcatHot.toarray()] # Faz a concatenação (np.c) do XnumLimpo com o XcatHot transformado em um array
Xtratado.shape

(891, 10)

In [15]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.columns.isin(test.columns) # Quais colunas tem na base de teste en relação a base de treino

array([ True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [16]:
train.columns[~train.columns.isin(test.columns)] # Mesmo comando anterior, porem selecionando as colunas que a base de treino nao tem

Index(['Survived'], dtype='object')

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin # Importa para utilizar na criação da classe (necessário para o cross validate?)

class AtributosDesejados(BaseEstimator, TransformerMixin): # Classe criada para retirar as colunas indesejadas de forma automática
    def fit(self, X, y=None):
        self.colunasIndesejadas = ['PassengerId', 'Name', 'Ticket', 'Cabin']
        return self
    def transform(self, X, y=None):
        return X.drop(self.colunasIndesejadas,axis=1)

atributosDesejados = AtributosDesejados() # Agora basta criar um objeto da classe
Xdrop = atributosDesejados.fit_transform(X) # E executar o transform
Xdrop.columns # As colunas idesejadas foram retiradas (mesma coisa que anteriormente, só que agora é reaproveitável)

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin
class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns # Seleciona de X somente as colunas com tipo numérico
        return self
    def transform(self, X, y=None):
        return X[self.colunasNumericas]

atributosNumericos = AtributosNumericos() # Agora basta criar um objeto da classe
Xnum = atributosNumericos.fit_transform(Xdrop) # E executar o transform
Xnum.columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

pipenum = Pipeline([ # Criando um pipeline para automatizar as tarefas dos valores numéricos em um conjunto de dados
    ('atributos_numericos', AtributosNumericos()), # Seleciona somente os valores numéricos
    ('imputer', SimpleImputer(strategy='median')), # Substitui os valores nulos com a mediana dos demais valores
    ('scaler', StandardScaler()) # Fazendo a padronização
])

XnumLimpo = pipenum.fit_transform(Xnum)
XnumLimpo

array([[ 0.82737724, -0.56573646,  0.43279337, -0.47367361, -0.50244517],
       [-1.56610693,  0.66386103,  0.43279337, -0.47367361,  0.78684529],
       [ 0.82737724, -0.25833709, -0.4745452 , -0.47367361, -0.48885426],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337,  2.00893337, -0.17626324],
       [-1.56610693, -0.25833709, -0.4745452 , -0.47367361, -0.04438104],
       [ 0.82737724,  0.20276197, -0.4745452 , -0.47367361, -0.49237783]])

In [20]:
from sklearn.base import BaseEstimator, TransformerMixin
class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(include='object').columns # Seleciona de X somente as colunas com tipo categórico (object)
        return self
    def transform(self, X, y=None):
        return X[self.colunasCategoricas]

atributosCategoricos = AtributosCategoricos() # Agora basta criar um objeto da classe
Xcat = atributosCategoricos.fit_transform(Xdrop) # E executar o transform
Xcat.columns # Agora será exibido somente as colunas categóricas

Index(['Sex', 'Embarked'], dtype='object')

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

pipecat = Pipeline([ # Criando um pipeline para automatizar as tarefas dos valores categóricos em um conjunto de dados
    ('atributos_categoricos', AtributosCategoricos()), # Seleciona somente os valores categóricos
    ('imputer', SimpleImputer(strategy='most_frequent')), # Substitui os valores nulos com os valores mais frequentes
    ('encoder', OneHotEncoder()) # Colocando o encoder
])

XcatLimpo = pipecat.fit_transform(Xdrop)
XcatLimpo

<891x5 sparse matrix of type '<class 'numpy.float64'>'
	with 1782 stored elements in Compressed Sparse Row format>

In [22]:
from sklearn.pipeline import FeatureUnion
unecaracteristicas = FeatureUnion([ # um pipeline para juntas tanto as características numéricas tratadas quanto as categóricas tratadas
    ('pipenum', pipenum), # Reutiliza os pipelines criados anteriormente
    ('pipecat', pipecat)
])
Xtratado = unecaracteristicas.fit_transform(Xdrop)
Xtratado

<891x10 sparse matrix of type '<class 'numpy.float64'>'
	with 6237 stored elements in Compressed Sparse Row format>

In [23]:
from sklearn.pipeline import Pipeline 

preproc = Pipeline([ # Agora o objetivo é receber um X quadrado e já sair totalmente redondo e pronto para usar
    ('atributos_desejados', AtributosDesejados()),
    ('unecaracteristicas', unecaracteristicas)
])
Xtratado = preproc.fit_transform(X)
Xtratado

<891x10 sparse matrix of type '<class 'numpy.float64'>'
	with 6237 stored elements in Compressed Sparse Row format>

In [24]:
from sklearn.tree import DecisionTreeClassifier # Usando um classificador em árvore

pipetotal = Pipeline([ # A intenção é fazer o pipeline geral dado os dados não tratados ele já faz a predição
    ('preproc', preproc),
    ('arvore', DecisionTreeClassifier())
])

In [25]:
from sklearn.metrics import accuracy_score

pipetotal.fit(X,y) # Faz o fit com X e y
ypred = pipetotal.predict(X)
accuracy_score(y, ypred) # Faz o processamento, mesmo que superotimista

0.9797979797979798

In [26]:
from sklearn.model_selection import cross_validate # Calculando a precisão com o cross_validate
import numpy as np

scores = cross_validate(pipetotal, X, y)
scores, np.mean(scores['test_score'])

({'fit_time': array([0.02499843, 0.02299929, 0.02700186, 0.02600193, 0.02700043]),
  'score_time': array([0.01000071, 0.01000237, 0.00799847, 0.00799823, 0.00800109]),
  'test_score': array([0.75977654, 0.78089888, 0.79775281, 0.73595506, 0.80898876])},
 0.7766744083861653)

In [27]:
from sklearn.model_selection import GridSearchCV # Calculando a precisão utilizando o melhor parâmetro (GridSearch)

parametros = {
    'arvore__max_depth': [None] + list(range(1,20,2)), # A profundidade máxima da árvore é por padrão vazio (None), mas vai de 1 à 20 de 2 em 2 (1,20,2)
    'arvore__criterion': ['gini', 'entropy'] # O critério a ser utilizado é o padrão "gini", mas vai testar também utilizando a "entropy"
}
modelo = GridSearchCV(pipetotal, param_grid=parametros)
scores = cross_validate(modelo, X, y)
scores, np.mean(scores['test_score'])

({'fit_time': array([3.38206029, 3.38500023, 3.32299042, 3.36205196, 3.56403995]),
  'score_time': array([0.00694036, 0.00699902, 0.00700569, 0.00599957, 0.00794911]),
  'test_score': array([0.82122905, 0.80337079, 0.81460674, 0.78089888, 0.82022472])},
 0.8080660347749671)

In [28]:
from sklearn.ensemble import RandomForestClassifier

clf = Pipeline([
    ('preproc', preproc),
    ('arvore', RandomForestClassifier())
])
clf.fit(X,y)
y_pred = clf.predict(test)

In [29]:
result = test[['PassengerId']]
result['Survived'] = y_pred
result.to_csv('videoaula.csv',index=False)

<ipython-input-29-3e0491657120>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Survived'] = y_pred
